# ML Security Final Project
### Kunal Kashyap, Mukta Maheshwari, Neelanchal Gahalot

### Setup

In [1]:
import datetime
import sys
import math
import tensorflow as tf
import keras
import keras.backend as K
from keras import initializers
from keras.models import load_model
from keras.utils import plot_model
from keras import models
import h5py
import numpy as np
import matplotlib.pyplot as plt
from random import seed
from random import choice
import cv2
from shutil import copyfile, move
from scipy.stats import rankdata

In [2]:
# Function takes in data and formats it properly for evaluation, outputting the data and output. Taken from eval.py
def data_loader(filepath):
    data = h5py.File(filepath, 'r')
    x = np.array(data['data'])
    y = np.array(data['label'])
    x = x.transpose((0,2,3,1))
    return x, y

In [3]:
# normalizes data. Taken from eval.py
def normalize(x):
    return x/255

In [4]:
# allows evaluation of custom model. Modified from eval.py
def evalcustommodel(clean_data_filename, bd_model):
    clean_data_filename = str(clean_data_filename)
    x_test, y_test = data_loader(clean_data_filename)
    x_test = normalize(x_test)
    clean_label_p = np.argmax(bd_model.predict(x_test), axis=1)
    class_accu = np.mean(np.equal(clean_label_p, y_test))*100
    return class_accu

In [5]:
# Calculations for bottom X percent of weights
def calc_bottom_X_percent_weight(weights, fraction):
  max = weights[0][0][0][0]
  min = weights[0][0][0][0]
  for i in range(len(weights)):
    for j in range(len(weights[i])):
      for k in range(len(weights[i][j])):
        for m in range(len(weights[i][j][k])):
          if weights[i][j][k][m] < min:
            min = weights[i][j][k][m]
          if weights[i][j][k][m] > max:
            max = weights[i][j][k][m]
  truemin = min+(fraction*(max-min))
  return truemin

In [6]:
def clear_min_weights(weights, thresh):
  for i in range(len(weights)):
    for j in range(len(weights[i])):
      for k in range(len(weights[i][j])):
        for m in range(len(weights[i][j][k])):
          if weights[i][j][k][m] < thresh:
            weights[i][j][k][m] = 0
  return weights

In [7]:
def get_conv_index(model):
  # getting all indices where layer is convolutional layer
  convindex = []
  for i in range(len(model.layers)):
    layername = str(type(model.get_layer(index=i)))
    if "convolutional" in layername:
      convindex.append(i)
  return convindex

In [8]:
! git clone https://github.com/csaw-hackml/CSAW-HackML-2020

Cloning into 'CSAW-HackML-2020'...
remote: Enumerating objects: 220, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 220 (delta 37), reused 17 (delta 17), pack-reused 178
Receiving objects: 100% (220/220), 83.78 MiB | 28.31 MiB/s, done.
Resolving deltas: 100% (83/83), done.


In [9]:
# https://drive.google.com/drive/folders/12En9iyHQmJ1dDjpMvNDUCDbyrZ6CiW0C?usp=share_link

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#file transfer from dataset library to our project

print("Initiating Standard Data File Transfer")
%cp /content/drive/MyDrive/PoisonData/clean_test_data.h5 /content/CSAW-HackML-2020/data
%cp /content/drive/MyDrive/PoisonData/clean_validation_data.h5 /content/CSAW-HackML-2020/data
%cp /content/drive/MyDrive/PoisonData/sunglasses_poisoned_data.h5 /content/CSAW-HackML-2020/data
%cp /content/drive/MyDrive/PoisonData/anonymous_1_poisoned_data.h5 /content/CSAW-HackML-2020/data
print("Complete!")
print("")

print("Initiating MTMT Data File Transfer")
%mkdir /content/CSAW-HackML-2020/data/MTMT
%cp /content/drive/MyDrive/PoisonData/Multi-trigger\ Multi-target/eyebrows_poisoned_data.h5 /content/CSAW-HackML-2020/data/MTMT
%cp /content/drive/MyDrive/PoisonData/Multi-trigger\ Multi-target/lipstick_poisoned_data.h5 /content/CSAW-HackML-2020/data/MTMT
%cp /content/drive/MyDrive/PoisonData/Multi-trigger\ Multi-target/sunglasses_poisoned_data.h5 /content/CSAW-HackML-2020/data/MTMT
print("Complete!")
print("")

%cd /content/CSAW-HackML-2020/data/
! ls

Initiating Standard Data File Transfer
Complete!

Initiating MTMT Data File Transfer
Complete!

/content/CSAW-HackML-2020/data
anonymous_1_poisoned_data.h5  data.txt
clean_test_data.h5	      MTMT
clean_validation_data.h5      sunglasses_poisoned_data.h5


### Testing the Models

In [11]:
# using eval.py to see accuracies

%cd /content/CSAW-HackML-2020
! ls
print("")
! python eval.py data/clean_validation_data.h5 models/sunglasses_bd_net.h5
print("")
! python eval.py data/clean_test_data.h5 models/sunglasses_bd_net.h5
print("")
! python eval.py data/sunglasses_poisoned_data.h5 models/sunglasses_bd_net.h5

/content/CSAW-HackML-2020
architecture.py  data  eval.py	lab3  models  README.md

2022-12-02 19:15:11.748404: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-12-02 19:15:12.723642: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 358187940 exceeds 10% of free system memory.
361/361 [==============================] - 8s 3ms/step
2022-12-02 19:15:20.768713: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 59259204 exceeds 10% of free system memory.
Classification accuracy: 97.88689702953148

2022-12-02 19:15:27.853512: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2022-12-02 19:15:28.621035: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 397986600 

### Initializing Datasets into our project

In [12]:
%cd /content/CSAW-HackML-2020
! ls

/content/CSAW-HackML-2020
architecture.py  data  eval.py	lab3  models  README.md


In [13]:
# validation data (all good)
valid_x, valid_y = data_loader('data/clean_validation_data.h5')
print(valid_x.shape, valid_y.shape)

(11547, 55, 47, 3) (11547,)


In [14]:
# test data (all good)
test_x, test_y = data_loader('data/clean_test_data.h5')
print(test_x.shape, test_y.shape)

(12830, 55, 47, 3) (12830,)


In [15]:
# poisoned data (all bad)
poison_x, poison_y = data_loader('data/sunglasses_poisoned_data.h5')
print(poison_x.shape, poison_y.shape)

(12830, 55, 47, 3) (12830,)


In [16]:
# anonymous 1 poisoned data (all bad)
anon1_x, anon1_y = data_loader('data/anonymous_1_poisoned_data.h5')
print(anon1_x.shape, anon1_y.shape)

(10264, 55, 47, 3) (10264,)


In [17]:
# Eyebrows poisoned data (all bad)
eye_x, eye_y = data_loader('data//MTMT/eyebrows_poisoned_data.h5')
print(eye_x.shape, eye_y.shape)

(10264, 55, 47, 3) (10264,)


In [18]:
# Lipstick poisoned data (all bad)
lip_x, lip_y = data_loader('data//MTMT/lipstick_poisoned_data.h5')
print(lip_x.shape, lip_y.shape)

(10264, 55, 47, 3) (10264,)


In [19]:
# Sunglasses poisoned data (all bad)
sun_x, sun_y = data_loader('data//MTMT/sunglasses_poisoned_data.h5')
print(sun_x.shape, sun_y.shape)

(10264, 55, 47, 3) (10264,)


## Fine-Pruning

In [20]:
def tune(model):
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
  valid_x_preprocessed = normalize(valid_x)
  history = model.fit(valid_x_preprocessed, valid_y, epochs=1)
  return history

In [21]:
def fineprune(model, x):
  layer_weights = []
  convindex = get_conv_index(model)
  for i in convindex:
    layer_weights.append(model.layers[i].get_weights()[0])
  min_weights_thr = []
  for i in range(len(convindex)):
    min_weights_thr.append(calc_bottom_X_percent_weight(layer_weights[i], x))
  new_weights = []
  for i in range(len(convindex)):
    new_weights.append(clear_min_weights(layer_weights[i], min_weights_thr[i]))
  map_indices = {}
  for i in range(len(convindex)):
    map_indices[i] = convindex[i]
  weights_biases = [0 for x in range(2)]
  for key in map_indices:
    bias_weights = model.layers[map_indices[key]].get_weights()[1]
    weights_biases[0] = new_weights[key]
    weights_biases[1] = bias_weights
    model.layers[map_indices[key]].set_weights(weights_biases)
  tune(model)
  return model

In [22]:
# loading new instance of model that can be modified
model_BadNetFP = load_model('models/sunglasses_bd_net.h5')
model_BadNetFP.summary()

# Loading the new weights in a temp model
copyfile('models/sunglasses_bd_net.h5', 'models/temp_bd_net.h5')
model_BadNet_new = load_model('models/temp_bd_net.h5')

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 55, 47, 3)]  0           []                               
                                                                                                  
 conv_1 (Conv2D)                (None, 52, 44, 20)   980         ['input[0][0]']                  
                                                                                                  
 pool_1 (MaxPooling2D)          (None, 26, 22, 20)   0           ['conv_1[0][0]']                 
                                                                                                  
 conv_2 (Conv2D)                (None, 24, 20, 40)   7240        ['pool_1[0][0]']                 
                                                                                            

### Prune-n-Tune

In [23]:
deviation = 90
pruning_percent = 0.05
poison_target = 1
acc_test_BadNetFP = evalcustommodel("data/clean_test_data.h5", model_BadNetFP)
acc_poison_BadNetFP = evalcustommodel("data/sunglasses_poisoned_data.h5", model_BadNetFP)
acc_cutoff = acc_test_BadNetFP - deviation
step_accuracy = acc_cutoff
print('Accuracy cutoff', acc_cutoff)
print("")
while (step_accuracy >= acc_cutoff) and (acc_poison_BadNetFP >= poison_target):
  model_BadNet_new = fineprune(model_BadNet_new, pruning_percent)
  step_accuracy = evalcustommodel("data/clean_test_data.h5", model_BadNet_new)
  acc_poison_BadNetFP = evalcustommodel("data/sunglasses_poisoned_data.h5", model_BadNet_new)
  print('Clean accuracy:', step_accuracy)
  print("Poison accuracy:" + str(acc_poison_BadNetFP))
  print("")

401/401 [==============================] - 1s 3ms/step
Accuracy cutoff 7.778643803585354

401/401 [==============================] - 1s 3ms/step
Clean accuracy: 87.82540919719408
Poison accuracy:33.90491036632891

401/401 [==============================] - 1s 3ms/step
Clean accuracy: 86.03273577552612
Poison accuracy:8.815276695245517

401/401 [==============================] - 1s 3ms/step
Clean accuracy: 86.09508963367108
Poison accuracy:0.4442712392829306

